In [110]:
from gurobipy import Model, GRB
import numpy as np
import pandas as pd
from calories_of_ingredients import CALORIES_OF_INGREDIENTS
from food_costs import FOOD_COSTS
from recipes import RECIPES

In [122]:
recipes = {}
calories_of_ingredients = {}
food_costs = {}
recipes = RECIPES
calories_of_ingredients = CALORIES_OF_INGREDIENTS
food_costs = FOOD_COSTS
max_num_times_recipe_is_eaten = {}
min_num_times_recipe_is_eaten = {}

In [123]:
#recipes['dish_name'] = {'ingredient_1_name' : grams, 'ingredient_2_name' : grams}
# recipes['potatoes_and_meat'] = {'potatoes' : 500, 'meat' : 300}
# recipes['eggs_and_carrots'] = {'eggs' : 120, 'carrots' : 200}

In [124]:
max_num_times_recipe_is_eaten['potatoes_and_meat'] = 30
max_num_times_recipe_is_eaten['eggs_and_carrots'] = 30
max_num_times_recipe_is_eaten['bacon_and_eggs'] = 30
min_num_times_recipe_is_eaten['potatoes_and_meat'] = 4
min_num_times_recipe_is_eaten['eggs_and_carrots'] = 4
min_num_times_recipe_is_eaten['bacon_and_eggs'] = 4

In [125]:
#ingredients['ingredient_name'] = amount in calories per gram
# calories_of_ingredients['potatoes'] = 0.6
# calories_of_ingredients['carrots'] = 0.7
# calories_of_ingredients['eggs'] = 0.8
# calories_of_ingredients['meat'] = 1.5

In [126]:
#food_costs['ingredient_name'] = dollars per gram
# food_costs['potatoes'] = 2
# food_costs['carrots'] = 1
# food_costs['eggs'] = 5
# food_costs['meat'] = 7

In [127]:
#Define params
num_meals_per_day = 3
num_days = 14
total_calories_per_day = 1800
calorie_tolerance = 50
time_intervals = [*range(num_meals_per_day*num_days)]

In [128]:
#Initialize model
model = Model()

In [129]:
#Define variables
amount_of_recipe = {}
for time_interval in time_intervals:
    amount_of_recipe[time_interval] = {}
    for recipe in recipes.keys():
        amount_of_recipe[time_interval][recipe] = 0
        
recipe_present = {}
for time_interval in time_intervals:
    recipe_present[time_interval] = {}
    for recipe in recipes.keys():
        recipe_present[time_interval][recipe] = 0

In [130]:
#Define variables as Gurobi variables
for time_interval in time_intervals:
    for recipe in recipes.keys():
        amount_of_recipe[time_interval][recipe] = model.addVar(0.5,
                                                               10, 
                                                               name = f'amount_of_recipe[{time_interval}][{recipe}]',
                                                               vtype = 'S')

for time_interval in time_intervals:
    for recipe in recipes.keys():
        recipe_present[time_interval][recipe] = model.addVar(name = f'recipe_present[{time_interval}][{recipe}]',
                                                             vtype = 'B')

In [131]:
#Define constraints
for time_interval in time_intervals:
    s = 0
    for recipe in recipes.keys():
        s += recipe_present[time_interval][recipe]
    model.addConstr(s == 1)

for time_interval in time_intervals:
    for recipe in recipes.keys():
        model.addConstr(recipe_present[time_interval][recipe] 
                        + amount_of_recipe[time_interval][recipe] <= 50*recipe_present[time_interval][recipe])
        model.addConstr(recipe_present[time_interval][recipe] 
                        + amount_of_recipe[time_interval][recipe] <= 50*amount_of_recipe[time_interval][recipe])

s = 0
for time_interval in time_intervals:
    for recipe in recipes.keys():
        for ingredient in recipes[recipe]:
            s += recipes[recipe][ingredient]*calories_of_ingredients[ingredient]*amount_of_recipe[time_interval][recipe]
    if (time_interval + 1) % num_meals_per_day == 0:
        model.addConstr(s <= total_calories_per_day + calorie_tolerance)
        model.addConstr(s >= total_calories_per_day - calorie_tolerance)
        s = 0
        
for recipe in recipes.keys():
    s = 0
    for time_interval in time_intervals:
        s += recipe_present[time_interval][recipe]
    model.addConstr(s <= max_num_times_recipe_is_eaten[recipe])
    model.addConstr(s >= min_num_times_recipe_is_eaten[recipe])

In [ ]:
#Set constraints for certain recipes


In [132]:
#Define objective function and optimize
def get_food_cost(amount_of_recipe):
    total_cost = 0
    for time_interval in time_intervals:
        for recipe in recipes.keys():
            for ingredient in recipes[recipe]:
                total_cost += recipes[recipe][ingredient]*amount_of_recipe[time_interval][recipe]*food_costs[ingredient]
                
    return total_cost

model.setObjective(get_food_cost(amount_of_recipe), GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 242 rows, 168 columns and 756 nonzeros
Model fingerprint: 0xe8c61e13
Variable types: 0 continuous, 84 integer (84 binary)
Semi-Variable types: 84 continuous, 0 integer
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [8e+02, 3e+03]
  Bounds range     [5e-01, 1e+01]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 92267.308475
Presolve removed 44 rows and 42 columns
Presolve time: 0.00s
Presolved: 366 rows, 210 columns, 924 nonzeros
Variable types: 84 continuous, 126 integer (126 binary)

Root relaxation: objective 8.318741e+04, 183 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 83187.4092    0   12 92267.3085 83187.4092  9.84%     -    0s
H    0  

In [139]:
for time_interval in time_intervals:
    for recipe in recipes.keys():
        if round(amount_of_recipe[time_interval][recipe].x, 2) != 0:
            print(f'time interval:', time_interval, 
                  'recipe:', recipe, 
                  'amount:', round(amount_of_recipe[time_interval][recipe].x, 2))
    if (time_interval + 1) % 3 == 0:
        print('-------------------------------------------------------')

time interval: 0 recipe: eggs_and_carrots amount: 4.24
time interval: 1 recipe: potatoes_and_meat amount: 0.5
time interval: 2 recipe: potatoes_and_meat amount: 0.5
-------------------------------------------------------
time interval: 3 recipe: eggs_and_carrots amount: 6.42
time interval: 4 recipe: eggs_and_carrots amount: 0.5
time interval: 5 recipe: eggs_and_carrots amount: 0.5
-------------------------------------------------------
time interval: 6 recipe: eggs_and_carrots amount: 4.24
time interval: 7 recipe: potatoes_and_meat amount: 0.5
time interval: 8 recipe: potatoes_and_meat amount: 0.5
-------------------------------------------------------
time interval: 9 recipe: eggs_and_carrots amount: 5.33
time interval: 10 recipe: potatoes_and_meat amount: 0.5
time interval: 11 recipe: eggs_and_carrots amount: 0.5
-------------------------------------------------------
time interval: 12 recipe: eggs_and_carrots amount: 5.33
time interval: 13 recipe: eggs_and_carrots amount: 0.5
time i

In [135]:
# for time_interval in time_intervals:
#     for recipe in recipes.keys():
#         print(f'time interval:', time_interval, 'recipe:', recipe, 'amount:', recipe_present[time_interval][recipe].x)